Train a LSTM network to with the training file. Validate the trained model with the valid file. Comment what you are doing in each part of your code. As the better the code, comments and result validation as the better the grade.

In [25]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer


In [26]:
# Load the data from the csv files
train_data = pd.read_csv('sent_train.csv')
valid_data = pd.read_csv('sent_valid.csv')
print(train_data.head(5))

# Preprocess the data (e.g., normalize, encode labels, etc.)
# Normalize the data
train_data = train_data.values
train_data = train_data[:, 1:]

# encode the labels
train_data[train_data == 'positive'] = 1
train_data[train_data == 'negative'] = 0

# Split the data into features and labels
X_train = train_data[:, 1:]
y_train = train_data[:, 0]

                                                text  label
0  $BYND - JPMorgan reels in expectations on Beyo...      0
1  $CCL $RCL - Nomura points to bookings weakness...      0
2  $CX - Cemex cut at Credit Suisse, J.P. Morgan ...      0
3  $ESS: BTIG Research cuts to Neutral https://t....      0
4  $FNKO - Funko slides after Piper Jaffray PT cu...      0


In [27]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Reshape the data
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_val = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(1, 300)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/opt/miniconda3/envs/uie-aa/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
